In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [6]:
from proj1_helpers import *
from implementations import *
import os
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'train.csv'
data_path = os.path.join(data_base_path, data_folder)
y, tX, ids = load_csv_data(data_path)

### Change later : Simple data description 
- all variables are floating point, except PRI_jet_num which is integer
- variables prefixed with PRI (for PRImitives) are “raw” quantities about the bunch collision as measured by the detector.
- variables prefixed with DER (for DERived) are quantities computed from the primitive features, which were selected by the physicists of ATLAS.
- it can happen that for some entries some variables are meaningless or cannot be computed; in this case, their value is −999.0, which is outside the normal range of all variables.

In [7]:
print('y shape = ' + str(y.shape) + '\ntX shape =' + str(tX.shape) + '\nids shape = ' + str(ids.shape))

y shape = (250000,)
tX shape =(250000, 30)
ids shape = (250000,)


### Data pre-processing

We will first transform the categorical variable (PRI_jet_num) into four dummy variables

In [ ]:
cX, fac_X = separate_factor(tX)
print(cX.shape, fac_X.shape)

filt_cX = missingness_filter(cX, 0.6)
print(filt_cX.shape)

In [1]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(arr, arr.shape)

NameError: name 'np' is not defined

--------------

In [ ]:
# putting the meaningless values to the median of the column
tX = np.where(tX==-999., np.nan,tX)
med_X = np.nanmedian(tX, axis=0)

inds = np.where(np.isnan(tX))
tX[inds] = np.take(med_X, inds[1])

In [ ]:
print(np.where(tX[0] == 2))

In [ ]:
# remove categorical data and standarize the rest
ntX = np.delete(tX, 22, axis=1)
ntX = np.apply_along_axis(standardize, 1, ntX)

### Make accuracy, recall and F1 functions

In [47]:
#prepare data and weights of some regression
poly, to_r = train_data_formatting(tX, degree = 5, cutoff = 0.1, imputation = impute_mean, interaction = False)
print(poly.shape, to_r)

(250000, 96) [ 0  4  5  6 12 23 24 25 26 27 28]


In [48]:
#make train and test data
poly_train = poly[0:150000,:]
poly_test = poly[150000:250000,:]
y_train = y[0:150000,]
y_test = y[150000:250000,]
print(poly_train.shape,poly_test.shape,y_train.shape,y_test.shape)

(150000, 96) (100000, 96) (150000,) (100000,)


In [49]:
#build simple least squares model
w, _ = least_squares(y_train, poly_train)
print(w.shape)

(96,)


In [51]:
y_train_pred = our_predict_labels(w, poly_train)
y_test_pred = our_predict_labels(w, poly_test)

In [53]:
compute_accuracy_measures(y_train, y_train_pred)

(0.7722, 0.6913687686883563, 0.603698219087331, 0.6445660314554381)

In [54]:
compute_accuracy_measures(y_test, y_test_pred)

(0.77145, 0.6917044847837693, 0.6035521910030572, 0.6446286131886243)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)